## Check dicom and mhd
---

### simple Reading / checking format

In [1]:
import pydicom
import SimpleITK

d = pydicom.read_file('./Dicom/L0/SE0/IM0')
m = SimpleITK.ReadImage('./mhd/L0.mhd')

# print(d)
print("Dicom Maker >> ", d.Manufacturer)
print("Dicom Shape >> ", d.pixel_array.shape)

# print(m)
print("mhd Shape >>", SimpleITK.GetArrayFromImage(m).shape)

Dicom Maker >>  GE MEDICAL SYSTEMS
Dicom Shape >>  (512, 512)
mhd Shape >> (193, 512, 512)


### dicom Reading(continuous) / checking lack

In [67]:
import glob
from natsort import natsorted
import traceback

# 同じディレクトリにあるdicomファイルを連続したデータに変換する
def Read_dicom_boxel(path):
    d_names = []
    d_names.extend(reversed(natsorted(glob.glob(path+"*"))))

    ref_dicom = pydicom.read_file(d_names[0])
    d_array   = np.zeros((ref_dicom.Rows, ref_dicom.Columns, len(d_names)), dtype=ref_dicom.pixel_array.dtype)
    
    for d_name in d_names:
        try:
            d = pydicom.read_file(d_name)
            d_array[:, :, d_names.index(d_name)] = d.pixel_array
        except: 
            #traceback.print_exc()
            print("err >> ", d_name)
            
    print('_'*5,'Read %s'%path,'_'*2, d_array.shape, '_'*5)
    return d_array


# 上から順に dicom mhd 教師データ
d_array = Read_dicom_boxel('./Dicom/L0/SE0/')
m_array = SimpleITK.GetArrayFromImage(m)                                       # ↓ 0が途切れ途切れ, 1が繋がっている
t_array = (SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(natsorted(glob.glob('./GroundTruth/L0/*.mhd'))[0])))*255

err >>  ./Dicom/L0/SE0/IM177
_____ Read ./Dicom/L0/SE0/ __ (512, 512, 193) _____


### Display dicom and mhd

In [70]:
from ipywidgets import interact
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 20), dpi=25)
plt.rcParams['figure.figsize'] = (15.0, 5.0)

@interact(slices=(0, d_array.shape[2]-1))
def plot_rolling_mean(slices):
    plt.subplot(131)
    plt.imshow(d_array[:, :, slices])
    plt.subplot(132)
    plt.imshow(m_array[slices, :, :])
    plt.subplot(133)
    plt.imshow(t_array[slices, :, :])
    
    plt.gray()
    plt.show()


<Figure size 500x500 with 0 Axes>

interactive(children=(IntSlider(value=96, description='slices', max=192), Output()), _dom_classes=('widget-int…

### display adipose

In [77]:
import copy

plt.rcParams['figure.figsize'] = (10.0, 5.0)

@interact(slices=(0, m_array.shape[0]-1), minimum=(np.min(m_array), np.max(m_array)),maximum=(np.min(m_array), np.max(m_array)))
def plot_adipose(slices,minimum=-250,maximum=-30):
    adipose = deepcopy(m_array)             # 論文内だと下記の範囲が脂肪(元は-190~-30)
    adipose[adipose < minimum] = 0          #  ---- | -250 | +++++++++++++++++++
    adipose[adipose > maximum] = 0          #  +++++++++++++++++++++| -30 | ----
    
    plt.subplot(121)
    plt.imshow(adipose[slices])    
    
    plt.subplot(122)
    plt.imshow(t_array[slices, :, :])


interactive(children=(IntSlider(value=96, description='slices', max=192), IntSlider(value=-250, description='m…

## ----------------------

## Read continuous slices
---

In [11]:
import os

def Canny_filter(use_array):
    # Cannyを使う際にdatatypeに少し注意 -> hconcatでも同じdatatype必要に
    return cv2.Canny(np.uint8(use_array),10,100)

def Laplacian_filter(use_array):
    # Laplacianはノイズがひどい
    return np.uint8(cv2.Laplacian(np.uint8(use_array), cv2.CV_32F))

def load_itk(filename):
    # Reads the image using SimpleITK
    itkimage = SimpleITK.ReadImage(filename)
    # 0-1なので255倍に / Convert the image to a  numpy array first and then shuffle the dimensions to get axis in the order z,y,x
    ct_scan = (SimpleITK.GetArrayFromImage(itkimage))*255
    # Read the origin of the ct_scan, will be used to convert the coordinates from world to voxel and vice versa.
    #origin = np.array(list(reversed(itkimage.GetOrigin())))
    # Read the spacing along each dimension
    #spacing = np.array(list(reversed(itkimage.GetSpacing())))

    return ct_scan


def Save_dicom2image(d_array):
    # make directory
    save_dir = dir_name.replace('Dicom','png')
    gt_path  = dir_name.replace('Dicom','GroundTruth')+'/*.mhd'
    
    # adjust dicom value(-2000~3000) to png value(0~256)
    d_array = d_array - np.min(d_array)
    d_array = (d_array / np.max(d_array) *256).astype('int64')
 
    try:
        if not os.path.exists(save_dir): os.makedirs(save_dir)
        
        gt_array = load_itk(natsorted(glob.glob(gt_path))[1])
        for slices in range(d_array.shape[2]):
            imgs = cv2.hconcat([ np.uint8(d_array[:,:,slices]), np.uint8(gt_array[slices,:,:]) ])
            cv2.imwrite(save_dir+'/'+str(slices)+'.png', cv2.cvtColor(imgs, cv2.COLOR_GRAY2BGR))
    except:                    
        #traceback.print_exc()
        print("err >> ", gt_path)
        

In [ ]:
dicom_dir = './Dicom/*'
gt_dir = './GroundTruth/*/*.mhd'


for dir_name in natsorted(glob.glob(dicom_dir)):
    Save_dicom2image(Read_dicom_boxel(dir_name+"/*/"))

## Make Train / Val directory

In [13]:
import shutil

root_dir = './'

if not os.path.exists(root_dir+'train'): os.makedirs(root_dir+'train')
if not os.path.exists(root_dir+'val'): os.makedirs(root_dir+'val')

for png_file_path in natsorted(glob.glob(root_dir+'png/*'))[:12]:
    for png_img_path in natsorted(glob.glob(png_file_path+"/*")):
        each_dir = png_img_path.split('/')
        shutil.copyfile(png_img_path, root_dir+'train/'+each_dir[-2]+'-'+each_dir[-1])
        
for png_file_path in natsorted(glob.glob(root_dir+'png/*'))[12:]:
    for png_img_path in natsorted(glob.glob(png_file_path+"/*")):
        each_dir = png_img_path.split('/')
        shutil.copyfile(png_img_path, root_dir+'val/'+each_dir[-2]+'-'+each_dir[-1])

## 今後すること
---

精度検証

データ拡張(2万枚くらいに)
- コントラスト変化
- ノイズ追加
- 

